In [ ]:
!mkdir gazeset_1
#!gdown https://drive.google.com/uc?id=1wqTA4gutC-L4h8TcMMQO_3jJYBL15-h3
!unzip -qq "/content/drive/MyDrive/data/UnityEye/UnityEye Archive.zip" -d gazeset_1

!mkdir gazeset
!unzip -qq "/content/drive/MyDrive/data/UnityEye/imgs50.zip" -d gazeset

!mkdir gazeset_2
!unzip -qq "/content/drive/MyDrive/eye_w/imgs.zip" -d gazeset_2

warning [/content/drive/MyDrive/eye_w/imgs.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [/content/drive/MyDrive/eye_w/imgs.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #201878).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)


In [ ]:
!ls /content/gazeset/imgs | wc -l

107969


In [ ]:
!find /content/gazeset_1/imgs -mindepth 1 -type f -name "*.json" -printf x | wc -c

10791


In [ ]:
!for FILENAME in $(ls /content/gazeset_1/imgs/); do mv /content/gazeset_1/imgs/$FILENAME /content/gazeset_2/imgs/1_$FILENAME; done

In [ ]:
!for FILENAME in $(ls /content/gazeset/imgs/); do mv /content/gazeset/imgs/$FILENAME /content/gazeset_2/imgs/2_$FILENAME; done

In [ ]:
!ls /content/gazeset_2/imgs | wc -l

331429


In [ ]:
!find /content/gazeset_2/imgs -mindepth 1 -type f -name "*.jpg" -printf x | wc -c

165713


In [ ]:
!rm -r eye

rm: cannot remove 'eye': No such file or directory


In [1]:
!mkdir eye
!git clone https://github.com/Etzelkut/Eye-Tracking.git /content/eye

Cloning into '/content/eye'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 302 (delta 193), reused 186 (delta 90), pack-reused 0
Receiving objects: 100% (302/302), 3.55 MiB | 13.33 MiB/s, done.
Resolving deltas: 100% (193/193), done.


In [2]:
!pip install local-attention
!pip install axial-positional-embedding
!pip install adabelief-pytorch
!pip install ranger-adabelief
!pip install pytorch-lightning
!pip install comet-ml
!pip install einops

  Created wheel for axial-positional-embedding: filename=axial_positional_embedding-0.2.1-py3-none-any.whl size=2901 sha256=975099c4a5eaa2d4ae67a848f383205a655e9ba230ac2abf7c9e7ed0d85f7c24
  Stored in directory: /root/.cache/pip/wheels/4a/2c/c3/9a1cb267c0d0d9b6eeba7952addb32b17857d1f799690c27a8
Successfully built axial-positional-embedding
     |████████████████████████████████| 527 kB 4.2 MB/s 
     |████████████████████████████████| 397 kB 51.2 MB/s 
     |████████████████████████████████| 596 kB 41.2 MB/s 
     |████████████████████████████████| 829 kB 46.6 MB/s 
     |████████████████████████████████| 133 kB 46.0 MB/s 
     |████████████████████████████████| 952 kB 46.9 MB/s 
     |████████████████████████████████| 1.1 MB 47.7 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 271 kB 37.6 MB/s 
     |████████████████████████████████| 144 kB 43.9 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 s

     |████████████████████████████████| 315 kB 4.2 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 548 kB 35.0 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34547 sha256=97dbb4796306b60cea662f6d0db8a9f5c1462c8a700947d04654965b84669313
  Stored in directory: /root/.cache/pip/wheels/0d/c4/19/13d74440f2a571841db6b6e0a273694327498884dafb9cf978
Successfully built configobj


In [3]:
from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
#import sys

In [4]:
from eye.basem.basic_dependency import *
from eye.gaze_track.pl_model_train import Gaze_Track_pl
from eye.gaze_track.datasets import Dataset_Unity_pl

In [ ]:
class CheckpointEveryNSteps(pl.Callback):
    """
    Save a checkpoint every N steps, instead of Lightning's default that checkpoints
    based on validation loss.
    """

    def __init__(
        self,
        save_step_frequency,
        prefix="N-Step-Checkpoint",
        use_modelcheckpoint_filename=False,
        pathh = False,
    ):
        """
        Args:
            save_step_frequency: how often to save in steps
            prefix: add a prefix to the name, only used if
                use_modelcheckpoint_filename=False
            use_modelcheckpoint_filename: just use the ModelCheckpoint callback's
                default filename, don't use ours.
        """
        self.save_step_frequency = save_step_frequency
        self.prefix = prefix
        self.use_modelcheckpoint_filename = use_modelcheckpoint_filename
        self.pathh = pathh

    def on_batch_end(self, trainer: pl.Trainer, _):
        """ Check if we should save a checkpoint after every train batch """
        epoch = trainer.current_epoch
        global_step = trainer.global_step
        if global_step % self.save_step_frequency == 0:
            if self.use_modelcheckpoint_filename:
                filename = trainer.checkpoint_callback.filename
            else:
                filename = f"{self.prefix}_{epoch}_{global_step}.ckpt"
            ckpt_path = os.path.join(self.pathh, filename)
            print("we are here!!!")
            trainer.save_checkpoint(ckpt_path)

In [ ]:
base_args = {
    "d": 0,
    "new_d": 256,
    "im_size": (96, 160),
    "patch_size": 16,
    "grayscale": True,
    "max_epochs": 30, 

    "halfing": False,

    "alternative_landmarks": True,
    "add_heatmap_loss": False,
}



########################


main_example_attention_block = {
    "type_module" : "gcn", #att, gcn, gcn_no_wn, linear, rpr, local_sa, glu_alt, light, dynamic
    "d": base_args["new_d"],
    "dropout": 0.05, # in some dropout do not exist
    "num_heads": 2, #if type_module == local_sa, then n_local_attn_heads + global = num_heads
    #params for local_sa
    "n_local_attn_heads": 0,
    "local_attn_window_size": 50,
    #params for glu
    "k_kernel_glu": 4,
    "downbot_glu": 2,
    #for rpr type
    "max_relative_positions_rpr": 8,
    #for dynamic and light
    #"kernel_size_dynamic": 30, 
    #for glu alt
    "alt_num_layers": 1,
    "alt_patch_size": 3,
    "alt_padding": 1,
}

add_example_attention_block = {
    "type_module" : "linear", #att, gcn, gcn_no_wn, linear, rpr, local_sa, glu_alt, #light, dynamic
    "d": base_args["new_d"],
    "dropout": 0.05, # in some dropout do not exist
    "num_heads": 2, #if type_module == local_sa, then n_local_attn_heads + global = num_heads
    #params for local_sa
    "n_local_attn_heads": 0,
    "local_attn_window_size": 50,
    #params for glu
    "k_kernel_glu": 4,
    "downbot_glu": 2,
    #for rpr type
    "max_relative_positions_rpr": 8,
    #for dynamic and light
    #"kernel_size_dynamic": 30, 
    #for glu alt
    "alt_num_layers": 1,
    "alt_patch_size": 3,
    "alt_padding": 1,
}


example_branch = {
    "type_module": "1d", #2d
    "d_model": base_args["new_d"],
    "out_d": 256,
}


example_evolved_encoder_hparams = {
    "d": base_args["new_d"],
    "number_of_main": 1,
    "main_attention_block": main_example_attention_block,
    "branched_conv": example_branch,
    
    "number_of_add": 1,
    "add_attention_block": add_example_attention_block,

    "norm_after_block":False,
}

#########################



example_attention_block = {
    "type_module" : "att", #att, gcn, gcn_no_wn, linear, rpr, local_sa, glu_alt, #light, dynamic
    "d": base_args["new_d"],
    "dropout": 0.05, # in some dropout do not exist
    "num_heads": 4, #if type_module == local_sa, then n_local_attn_heads + global = num_heads
    #params for local_sa
    "n_local_attn_heads": 2,
    "local_attn_window_size": 32,
    #params for glu
    "k_kernel_glu": 4,
    "downbot_glu": 2,
    #for rpr type
    "max_relative_positions_rpr": 8,
    #for dynamic and light
    #"kernel_size_dynamic": 30, 
    #for glu alt
    "alt_num_layers": 1,
    "alt_patch_size": 3,
    "alt_padding": 1,
}


example_encoder_hparams = {
    "attention_block": example_attention_block,
    "d": base_args["new_d"],
    "ff_type": "fc", #fc, conv
    "ff_activation_type": "mish", #swish or gelu
    "ff_glu": False,
    "ff_dropout": 0.05,
    # encoder block params
    "layers_number":2,
    "norm_after_block":False,
    "alternative_weight_init": False,
}


example_emb = {
    "add_emb": False, # if false will run normal embeding from the ViT
    "emb_module": {
        "type_module": "axial", # const
        "d": base_args["new_d"], # dimensions
        "dropout": 0.1,
    }
}


example_resize = {
    "add_resize": True, 
    "resize_module": {
        "type_module": "fc", #fc, 1dconv
        "size": base_args["d"], # dimensions
        "new_size": base_args["new_d"],
    },
}


example_feature_extract_hparams = {
    "grayscale": base_args["grayscale"],
    "im_size": base_args["im_size"],
    "patch_size": base_args["patch_size"],
    "d_model_emb": base_args["new_d"],

    "resize_": example_resize,
    "pos_emb": example_emb,
    "dropout": 0.05,
    "number_of_learn_params": 1,
    "encoder_type": "transformer", # transformer, evolved
    "encoder_params": example_encoder_hparams, #example_evolved_encoder_hparams,

    "alternative_landmarks": base_args["alternative_landmarks"],

    "add_additional_train_landmarks": True,

    "halfing": base_args["halfing"],
    "add_pool_end": False,
}



train_hparams_example = {
    "optimizer": "adamW", # "belief", "ranger_belief", "adam", adamW
    "lr": 3e-4, #
    "epochs": base_args['max_epochs'], #
    #
    "add_sch": False,
    #
    #belief
    "eplison_belief": 1e-16,
    "beta": [0.9, 0.999], # not used
    "weight_decouple": True, 
    "weight_decay": 1e-4,
    "rectify": True,

}

dataset_hparams = {
    "img_dir": "./gazeset_2/imgs",
    "grayscale": base_args["grayscale"],
    "im_size": base_args["im_size"],
    "batch_size": 64,
    "num_workers":2,
    "dataloader_shuffle": True,

    "halfing": base_args["halfing"],

    "alternative_landmarks": base_args["alternative_landmarks"],
}

example_model_hparams = {
    "type": "trans_based", # not used
    "feature_extractor_hparams": example_feature_extract_hparams,

    "alternative_landmarks": base_args["alternative_landmarks"],
    "add_heatmap_loss": base_args["add_heatmap_loss"],
    "im_size": base_args["im_size"],

    "dataset_h": dataset_hparams,

    "mlp_drop": 0.05,
    "gaze_size": 2,
    "training": train_hparams_example,
}

In [ ]:
seed_v = 42
seed_everything(seed_v)

root_dir = "/content/drive/MyDrive/eye_w/weights"
naming = "trans_2_3e4_att_256_1learnparam_noNorm_land_alt_Mdata_"
from datetime import datetime

comet_logger = CometLogger(
  save_dir='/content/log/',
    api_key="23CU99n7TeyZdPeegNDlQ5aHf",
    project_name="gaze2",
    workspace="etzelkut",
  # rest_api_key=os.environ["COMET_REST_KEY"], # Optional
  experiment_name = naming, # Optional
)


#
dataset_pl = Dataset_Unity_pl(dataset_hparams)
dataset_pl.prepare_data()
dataset_pl.setup()
steps_per_epoch = int(len(dataset_pl.train_dataloader()))

every_epoch = CheckpointEveryNSteps(save_step_frequency = steps_per_epoch*2, 
                                    use_modelcheckpoint_filename = False, 
                                    pathh=root_dir, prefix=naming+"N-Step-Checkpoint")


proj_a = Gaze_Track_pl(example_model_hparams, steps_per_epoch = steps_per_epoch)


trainer = Trainer(callbacks=[every_epoch],
                  logger=comet_logger,
                  gpus=1,
                  profiler='simple',
                  #auto_lr_find=True, #set hparams
                  #gradient_clip_val=0.5,
                  check_val_every_n_epoch=1,
                  #early_stop_callback=True,
                  max_epochs = example_model_hparams["training"]["epochs"],
                  progress_bar_refresh_rate = 0,
                  deterministic=True,)

#~
trainer.fit(proj_a, dataset_pl)
trainer.test(proj_a, dataset_pl)

#if re_dict["training"]["swa"]:
#  proj_a.change_for_swa(dataset_pl.train_dataloader())
#  trainer.test() # will it work?
#.

checkpoint_name = os.path.join(root_dir, naming + '.ckpt')
trainer.save_checkpoint(checkpoint_name)
#copied with time for safety, not memory effec. but why bother I have infinite google drive
checkpoint_name = os.path.join(root_dir, naming + str(datetime.now()) + '.ckpt')
trainer.save_checkpoint(checkpoint_name)

In [ ]:
#https://arxiv.org/pdf/2007.13120v1.pdf
checkpoint_name = os.path.join(root_dir, naming + '.ckpt')
trainer.save_checkpoint(checkpoint_name)

In [ ]:
################################################################

In [ ]:
seed_v = 42
seed_everything(seed_v)

root_dir = "/content/drive/MyDrive/eye_w/weights"
naming = "trans_2_3e4_att_256_1learnparam_noNorm_land_alt_Mdata"
from datetime import datetime

comet_logger = CometLogger(
  save_dir='/content/log/',
    api_key="23CU99n7TeyZdPeegNDlQ5aHf",
    project_name="gaze2",
    workspace="etzelkut",
  # rest_api_key=os.environ["COMET_REST_KEY"], # Optional
  experiment_name = naming, # Optional
)


#

load_name = "/content/drive/MyDrive/eye_w/weights/trans_2_3e4_att_256_1learnparam_noNorm_land_alt_Mdata_N-Step-Checkpoint_26_53872.ckpt"

proj_a = Gaze_Track_pl.load_from_checkpoint(load_name)

dataset_hparams = proj_a.hparams["dataset_h"]

dataset_pl = Dataset_Unity_pl(dataset_hparams)
dataset_pl.prepare_data()
dataset_pl.setup()
steps_per_epoch = int(len(dataset_pl.train_dataloader()))

every_epoch = CheckpointEveryNSteps(save_step_frequency = steps_per_epoch, 
                                    use_modelcheckpoint_filename = False, 
                                    pathh=root_dir, prefix=naming+"N-Step-Checkpoint")



trainer = Trainer(callbacks=[every_epoch],
                  logger=comet_logger,
                  gpus=1,
                  profiler='simple',
                  #auto_lr_find=True, #set hparams
                  #gradient_clip_val=0.5,
                  check_val_every_n_epoch=1,
                  #early_stop_callback=True,
                  max_epochs = proj_a.hparams["training"]["epochs"] + 1,
                  progress_bar_refresh_rate = 0,
                  deterministic=True,
                  resume_from_checkpoint=load_name)

#~
trainer.fit(proj_a, dataset_pl)
trainer.test(proj_a, dataset_pl)

#if re_dict["training"]["swa"]:
#  proj_a.change_for_swa(dataset_pl.train_dataloader())
#  trainer.test() # will it work?
#.

checkpoint_name = os.path.join(root_dir, naming + '.ckpt')
trainer.save_checkpoint(checkpoint_name)
#copied with time for safety, not memory effec. but why bother I have infinite google drive
checkpoint_name = os.path.join(root_dir, naming + str(datetime.now()) + '.ckpt')
trainer.save_checkpoint(checkpoint_name)

In [ ]:
############################################